# Import

In [ ]:
import google.generativeai as genai
from pydantic import BaseModel, Field, field_validator, model_validator
import os
import random
from typing import List, Literal, Optional, Dict, Any
import typing_extensions as typing
from IPython.display import Markdown, display

# Konfiguration des API-Schlüssels
api_key = os.environ.get("API_KEY")
if not api_key:
    raise ValueError("API_KEY Umgebungsvariable ist nicht gesetzt.")
genai.configure(api_key=api_key)

##### Auswahl des Models
# Auswahl des Modells durch Benutzer oder Umgebung
model_speed = {
    'dumb': 'gemini-1.5-flash-8b',
    'fast': 'gemini-1.5-flash-latest',
    'clever': 'gemini-1.5-pro-latest'
}
model_type = os.getenv("MODEL_TYPE", model_speed['fast'])
print("Model:", model_type)


##### Import des Yijing Textes

In [ ]:
from pathlib import Path

yijing_txt_path = Path('yijing/resources/yijing.txt')

# read the text line for line
def read_yijing_lines(yijing_txt_path=yijing_txt_path):
    with yijing_txt_path.open() as f:
        return f.readlines()
    
def read_yijing_txt(yijing_txt_path=yijing_txt_path):
    with yijing_txt_path.open() as f:
        return f.read()

yijing_txt = read_yijing_txt()
yijing_chapters = read_yijing_lines()
yijing_chapters[:5]

In [ ]:

yijing_chapters[0]

In [ ]:
yijing_raw = yijing_chapters[0]
yijing_raw

##### yijing_processed

In [ ]:
yijing_processed = """
{
  "hexagram": {
    "name": "GUAI / DER DURCHBRUCH",
    "subtitle": "Die Entschlossenheit",
    "trigrams": {
      "above": {
        "name": "Dui",
        "attributes": "das Heitere, der See"
      },
      "below": {
        "name": "Kien",
        "attributes": "das Schöpferische, der Himmel"
      }
    },
    "meaning": {
      "description": "Das Zeichen bedeutet einerseits einen Durchbruch nach lange angesammelter Spannung, wie den Durchbruch eines geschwellten Flusses durch seine Dämme, wie einen Wolkenbruch. Auf menschliche Verhältnisse übertragen, ist es andererseits die Zeit, da allmählich die Gemeinen im Schwinden sind. Ihr Einfluß ist im Abnehmen, und durch eine entschlossene Aktion kommt eine Änderung der Verhältnisse zum Durchbruch.",
      "season": "dritter Monat (April-Mai)"
    }
  },
  "judgment": {
    "description": "Der Durchbruch. Entschlossen muß man am Hof des Königs die Sache bekanntmachen. Der Wahrheit gemäß muß sie verkündet werden. Gefahr! Man muß seine eigene Stadt benachrichtigen. Nicht fördernd ist es, zu den Waffen zu greifen. Fördernd ist es, etwas zu unternehmen.",
    "analysis": [
      "Leidenschaft und Vernunft können nicht zusammen bestehen, daher ist ein unbedingter Kampf notwendig.",
      "Entschlossenheit muß auf Stärke und Freundlichkeit beruhen.",
      "Kompromisse mit dem Schlechten sind nicht möglich.",
      "Der Kampf darf nicht direkt durch Gewalt geführt werden.",
      "Der Edle beginnt bei sich selbst, um das Böse zu entwaffnen."
    ]
  },
  "image": {
    "description": "Der See ist an den Himmel emporgestiegen: das Bild des Durchbruchs.",
    "lesson": "Der Edle spendet Reichtum nach unten hin und scheut es, bei seiner Tugend zu verweilen.",
    "warning": "Sammeln führt zu Zerstreuen; rechtzeitige Vorbereitung kann einem gewaltsamen Zusammenbruch vorbeugen."
  },
  "lines": [
    {
      "position": "Anfangs eine",
      "text": "Mächtig in den vorwärtsschreitenden Zehen. Geht man hin und ist der Sache nicht gewachsen, so macht man einen Fehler.",
      "interpretation": "Zu Beginn ist entschlossenes Voranschreiten schwierig. Blindes Draufgängertum führt zu unheilvollen Folgen."
    },
    {
      "position": "Neun auf zweitem Platz",
      "text": "Alarmruf. Abends und nachts Waffen. Fürchte nichts.",
      "interpretation": "Vorsicht und Wachsamkeit schützen vor Gefahren. Besonnenheit ist der rechte Weg zur Sicherheit."
    },
    {
      "position": "Neun auf drittem Platz",
      "text": "Mächtig in den Backenknochen zu sein bringt Unheil.",
      "interpretation": "Entschlossenheit ist notwendig, aber äußere Stärke zur falschen Zeit kann die Lage verschlimmern."
    },
    {
      "position": "Neun auf viertem Platz",
      "text": "An den Oberschenkeln ist keine Haut, und das Gehen fällt schwer.",
      "interpretation": "Eigensinn führt zu Konflikten. Würde man Ratschläge annehmen, könnte alles gutgehen."
    },
    {
      "position": "Neun auf fünftem Platz",
      "text": "Dem Unkraut gegenüber braucht es feste Entschlossenheit.",
      "interpretation": "Hindernisse müssen mit Entschlossenheit überwunden werden, ohne vom Weg abzukommen."
    },
    {
      "position": "Oben eine Sechs",
      "text": "Kein Ruf! Schließlich kommt Unheil.",
      "interpretation": "Nachlässigkeit beim Entfernen des Bösen führt zu erneutem Übel. Gründliche Arbeit ist notwendig."
    }
  ]
}

"""

##### HEXAGRAM_PROMPT

In [ ]:
HEXAGRAM_PROMPT = """
Du bist ein I Ging-Experte mit der Aufgabe, Hexagramm-Texte zu analysieren und in ein spezifisches JSON-Format zu überführen.

Wandle den Eingabetext in folgendes Format um:

{
  "hexagram": {
    "name": "Name des Hexagramms (z.B. 'GUAI / DER DURCHBRUCH')",
    "subtitle": "Untertitel oder Kernbedeutung",
    "trigrams": {
      "above": {
        "name": "Name des oberen Trigramms",
        "attributes": "Eigenschaften des Trigramms"
      },
      "below": {
        "name": "Name des unteren Trigramms",
        "attributes": "Eigenschaften des Trigramms"
      }
    },
    "meaning": {
      "description": "Hauptbedeutung des Hexagramms",
      "season": "Zugeordnete Jahreszeit"
    }
  },
  "judgment": {
    "description": "Der Urteilstext",
    "analysis": [
      "Liste von Analysepunkten zum Urteil"
    ]
  },
  "image": {
    "description": "Beschreibung des Bildes",
    "lesson": "Lehre für den Edlen",
    "warning": "Warnung oder zusätzliche Hinweise"
  },
  "lines": [
    {
      "position": "Position der Linie (z.B. 'Anfangs eine')",
      "text": "Text der Linie",
      "interpretation": "Interpretation der Linie"
    }
  ]
}

Wichtige Hinweise:
1. Extrahiere alle relevanten Informationen aus dem Eingabetext
2. Behalte die originale Formulierung wo möglich bei
3. Stelle sicher, dass alle Pflichtfelder gefüllt sind
4. Das lines-Array muss genau 6 Einträge enthalten
5. Füge fehlende Informationen mit "Keine Information verfügbar" ein

Analysiere nun den folgenden Hexagramm-Text und gib ihn im spezifizierten JSON-Format zurück:
"""

## Type definitions for the JSON output

In [ ]:
from typing import List
from typing_extensions import TypedDict, NotRequired

class Trigram(TypedDict):
    name: str
    attributes: str
    element: NotRequired[str]  # Optional: z. B. Holz, Feuer, Erde, Metall, Wasser
    symbolism: NotRequired[str]  # Optional: Symbolische Bedeutung des Trigramms

class Trigrams(TypedDict):
    above: Trigram
    below: Trigram

class Meaning(TypedDict):
    description: str
    season: NotRequired[str]  # Optional: Jahreszeit (z. B. "Frühling", "Sommer")
    symbols: NotRequired[List[str]]  # Optional: Schlüsselwörter oder Symbole
    context: NotRequired[str]  # Optional: Allgemeiner Kontext oder Bedeutung

class Judgment(TypedDict):
    description: str
    analysis: List[str]  # Analyse oder Kommentare zum Urteil
    advice: NotRequired[str]  # Optional: Handlungsempfehlungen aus dem Urteil

class Image(TypedDict):
    description: str
    lesson: str
    warning: NotRequired[str]  # Optional: Warnungen oder Vorsichtsmaßnahmen
    additional_notes: NotRequired[str]  # Optional: Zusätzliche Kommentare oder Metaphern

class Line(TypedDict):
    position: int  # Position der Linie (1-6)
    text: str  # Originaltext der Linie
    interpretation: str  # Interpretation der Linie
    changing: NotRequired[bool]  # Optional: Ob die Linie sich wandelt
    additional_context: NotRequired[str]  # Optional: Zusätzliche Erläuterungen

class Transformation(TypedDict):
    changing_lines: List[int]  # Liste der sich wandelnden Linien (1-6)
    resulting_hexagram: int  # Nummer des resultierenden Hexagramms

class HistoricalContext(TypedDict):
    summary: str  # Zusammenfassung des historischen Kontexts
    references: NotRequired[List[str]]  # Optional: Literatur oder Verweise

class HexagramCore(TypedDict):
    number: int  # Hexagrammnummer (1-64)
    name: str
    subtitle: str  # Zusätzlicher Titel oder Beschreibung
    trigrams: Trigrams  # Oberes und unteres Trigramm
    meaning: Meaning  # Allgemeine Bedeutung

class HexagramInfo(TypedDict):
    hexagram: HexagramCore
    judgment: Judgment
    image: Image
    lines: List[Line]  # Sechs Linien mit Position, Text und Interpretation
    transformation: NotRequired[Transformation]  # Optional: Wandlungsdetails
    historical_context: NotRequired[HistoricalContext]  # Optional: Historische Kommentare oder Kontexte


In [ ]:

from typing_extensions import TypedDict

class Trigram(TypedDict):
    name: str
    attributes: str

class Meaning(TypedDict):
    description: str
    season: str

class Judgment(TypedDict):
    description: str
    analysis: List[str]

class Image(TypedDict):
    description: str
    lesson: str
    warning: str

class Line(TypedDict):
    position: str
    text: str
    interpretation: str

class Trigrams(TypedDict):
    above: Trigram
    below: Trigram

class HexagramCore(TypedDict):
    name: str
    subtitle: str
    trigrams: Trigrams
    meaning: Meaning

class HexagramInfo(TypedDict):
    hexagram: HexagramCore
    judgment: Judgment
    image: Image
    lines: List[Line]


## Funktionen

##### process_hexagram

In [ ]:
def process_hexagram(text: str, model: genai.GenerativeModel) -> HexagramInfo:
    """
    Processes a single hexagram text and returns structured data
    """
    prompt = HEXAGRAM_PROMPT + text
    result = model.generate_content(
        prompt,
        generation_config=genai.GenerationConfig(
            response_mime_type="application/json",
            response_schema=HexagramInfo
        ),
    )
    return result.candidates[0].content

# Inirialisierung

##### model.start_chat()

In [ ]:

# Erstellen des Modells
model = genai.GenerativeModel(
    model_type,
    system_instruction=instruction
)

chat = model.start_chat()
# Beispielaufruf
hexagram_text = yijing_chapters[12]
result = chat.send_message(hexagram_text)
line = yijing_chapters[12]
line

# Überprüfen, ob die Antwort Teile enthält, um Fehler zu vermeiden
if hasattr(result, 'parts') and len(result.parts) > 0:
    print(result.parts[0])
    display(Markdown(result.parts[0].text))
else:
    print("Keine Antwortteile gefunden.")

# json export
import json
from google.ai.generativelanguage_v1beta.types import content

import json
import pandas as pd

# Zugriff auf die 'parts'-Liste und das erste 'ContentPart'-Objekt
json_text = result.parts[0].text  # Korrekt: Nutze Punktnotation statt Index-Operator

# Load the JSON into a Python dictionary
formated_text = json_text.strip('```json\n').strip('```')

data = json.loads(formated_text)
data

In [ ]:
# Convert the "lines" part of the JSON into a pandas DataFrame
lines_df = pd.DataFrame(data['lines'])


In [ ]:
lines_df